In [7]:
import pandas as pd 
import numpy as np
import itertools
import multiprocessing as mp

In [ ]:
dimension = 10
widths = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
matrix = [
    [0, 30, 17, 11, 24, 25, 24, 17, 16, 22],
    [0, 0, 21, 23, 26, 24, 27, 19, 11, 32],
    [0, 0, 0, 24, 18, 23, 31, 36, 28, 19],
    [0, 0, 0, 0, 19, 18, 33, 25, 20, 28],
    [0, 0, 0, 0, 0, 15, 37, 27, 17, 16],
    [0, 0, 0, 0, 0, 0, 27, 23, 29, 24],
    [0, 0, 0, 0, 0, 0, 0, 27, 31, 24],
    [0, 0, 0, 0, 0, 0, 0, 0, 14, 18],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 24],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          ]

In [5]:
def evaluation(dimension, widths, matrix):
    result = 0
    for i in range(0, dimension):
        for j in range(i + 1, dimension):
            sumWidths = 0
            for k in range(i, j+1):
                sumWidths += widths[k]
            d = (widths[i] + widths[j])/2 + sumWidths
            result += d * matrix[i][j]
    return result

In [6]:
evaluation(dimension, widths, matrix)

5883.0

In [ ]:
# Worker function for parallel execution
def worker(permutations_chunk, dimension, widths, matrix):
    min_distance = float('inf')
    best_layout = None

    for perm in permutations_chunk:
        distance = evaluation(dimension, widths, matrix, perm)
        if distance < min_distance:
            min_distance = distance
            best_layout = perm

    return best_layout, min_distance

# Divide permutations into chunks for parallel processing
def divide_chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def parallel_srflp(dimension, widths, matrix):
    facilities = list(range(dimension))
    # Generate all possible layouts (permutations)
    all_permutations = list(itertools.permutations(facilities))

    # Split the permutations into chunks for parallel processing
    num_chunks = mp.cpu_count()
    chunk_size = len(all_permutations) // num_chunks
    permutation_chunks = list(divide_chunks(all_permutations, chunk_size))

    # Create a multiprocessing pool
    with mp.Pool(processes=num_chunks) as pool:
        # Map the worker function across the chunks
        results = pool.starmap(worker, [(chunk, dimension, widths, matrix) for chunk in permutation_chunks])

    # Find the best result among all chunks
    best_layout, min_distance = min(results, key=lambda x: x[1])

    return best_layout, min_distance

# Execute the parallel SRFLP solver
best_layout, min_distance = parallel_srflp(dimension, widths, matrix)
print("Best Layout:", best_layout)
print("Minimum Weighted Distance:", min_distance)